# Short-Term Memory with LangGraph

This notebook demonstrates how to implement short-term memory in agents using LangGraph. Short-term memory captures the conversation history within a single session, allowing the agent to reference recent messages and maintain context across multiple turns.

## Key Concepts:
- **Short-term memory**: Stores conversation history in the current session
- **Message history**: Maintains all messages exchanged between user and agent
- **Context window**: Limited to current conversation only
- **Stateful agent**: Uses graph state to persist memory across steps

In [18]:
# Install required dependencies
import subprocess
import sys

packages = [
    "langgraph",
    "langchain",
    "langchain-openai",
    "python-dotenv"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("✓ All dependencies installed")

✓ All dependencies installed


In [19]:
import os
from dotenv import load_dotenv
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing_extensions import TypedDict

# Verify API key
api_key = " "
if not api_key:
    print("⚠️  Warning: OPENAI_API_KEY not found. Please set it in your .env file")
else:
    print("✓ OpenAI API key loaded")

✓ OpenAI API key loaded


## Step 1: Define the Agent State

The state contains:
- `messages`: List of all messages in the conversation (both user and AI)
- This represents our short-term memory buffer

In [20]:
from typing import Callable

# Define reducer function for messages
def add_messages(left: list[BaseMessage], right: list[BaseMessage]) -> list[BaseMessage]:
    """Append new messages to existing messages."""
    return left + right

# Define the agent state
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

print("✓ Agent state defined with short-term memory (message history)")

✓ Agent state defined with short-term memory (message history)


## Step 2: Initialize the LLM and Create Node Functions

The `agent` node will process the conversation history and generate responses.

In [22]:
try:
    # Initialize the LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, openai_api_key=api_key)
    
    # Define the agent node
    def agent(state: AgentState) -> dict:
        """
        Agent node that processes messages and generates responses.
        Uses short-term memory (all messages in state) as context.
        """
        # All messages are available in state.messages (short-term memory)
        messages = state["messages"]
        
        # Call the LLM with the full message history
        response = llm.invoke(messages)
        
        # Return the new message to be added to state
        return {"messages": [response]}
    
    print("✓ Agent node created successfully")

except Exception as e:
    print(f"✗ Error initializing LLM: {e}")
    print("Make sure OPENAI_API_KEY is set in your .env file")

✓ Agent node created successfully


## Step 3: Build the Graph

Create a simple linear graph where:
1. User adds a message
2. Agent processes it
3. Loop back to accept new messages

In [23]:
# Create the graph
graph_builder = StateGraph(AgentState)

# Add nodes
graph_builder.add_node("agent", agent)

# Add edges
graph_builder.add_edge(START, "agent")
graph_builder.add_edge("agent", END)

# Compile the graph
graph = graph_builder.compile()

print("✓ Graph compiled successfully")

✓ Graph compiled successfully


## Step 4: Run a Multi-Turn Conversation Demo

Demonstrate how the agent remembers context across multiple turns.

In [25]:
try:
    # Initialize conversation
    initial_state = {"messages": []}
    
    # Multi-turn conversation
    conversation_turns = [
        "My name is Alice and I'm learning about AI.",
        "What's my name?",
        "What was the first thing I told you?",
        "Can you summarize our conversation so far?"
    ]
    
    print("=" * 60)
    print("SHORT-TERM MEMORY DEMONSTRATION")
    print("=" * 60)
    print()
    
    current_state = initial_state.copy()
    
    for i, user_input in enumerate(conversation_turns, 1):
        print(f"Turn {i}: User Input")
        print(f">> {user_input}")
        print()
        
        # Add user message to state
        current_state["messages"].append(HumanMessage(content=user_input))
        
        # Show current short-term memory size
        print(f"📊 Short-term memory size: {len(current_state['messages'])} messages")
        print()
        
        # Run the agent
        result = graph.invoke(current_state)
        current_state = result
        
        # Extract and display agent response
        last_message = current_state["messages"][-1]
        print(f"Turn {i}: Agent Response")
        print(f"<< {last_message.content}")
        print()
        print("-" * 60)
        print()
    
    print("✓ Multi-turn conversation completed successfully")
    print(f"✓ Total messages in short-term memory: {len(current_state['messages'])}")

except Exception as e:
    print(f"✗ Error running conversation: {e}")
    import traceback
    traceback.print_exc()

SHORT-TERM MEMORY DEMONSTRATION

Turn 1: User Input
>> My name is Alice and I'm learning about AI.

📊 Short-term memory size: 1 messages

Turn 1: Agent Response
<< Hi Alice! It's great to hear that you're learning about AI. There's a lot to explore in this field, from machine learning and natural language processing to robotics and neural networks. Do you have any specific topics or questions in mind that you'd like to discuss?

------------------------------------------------------------

Turn 2: User Input
>> What's my name?

📊 Short-term memory size: 3 messages

Turn 2: Agent Response
<< Your name is Alice! How can I assist you further, Alice?

------------------------------------------------------------

Turn 3: User Input
>> What was the first thing I told you?

📊 Short-term memory size: 5 messages

Turn 3: Agent Response
<< The first thing you told me was that your name is Alice and that you're learning about AI. If you have more questions or topics you'd like to explore, feel fr

## Key Takeaways: Short-Term Memory Pattern

### How it Works:
1. **State accumulates messages**: Each new message is appended to the messages list in state
2. **Full context available**: The agent always has access to all messages in the current session
3. **Reducer function**: The `add_messages` function ensures messages are properly accumulated
4. **Stateless graph execution**: Each invoke processes from the current state

### Characteristics:
- ✅ Remembers all messages within a session
- ✅ Allows agent to reference any previous message
- ✅ Simple to implement
- ✅ Cost increases with conversation length
- ❌ No persistence across sessions
- ❌ Memory grows indefinitely (may hit token limits)

### Common Use Cases:
- Chat applications within a single session
- Question-answering over recent context
- Clarification and follow-up handling
- Interactive problem-solving